In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['ml_outcomes'] = df['ml_outcomes_div'].round(2)


In [3]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [4]:
FRAC_ADMIT

0.5

In [5]:
len(df)

100000

In [6]:
#df['ml_outcomes'] = df['R']

In [7]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','ml_outcomes_black','ml_outcomes_white','ml_outcomes_decision','B_p']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,ml_outcomes_black,ml_outcomes_white,ml_outcomes_decision,B_p
87320,0,15,0.04,9,15,-3.823794,-3.135931,0,0
39625,0,16,0.04,16,16,-3.272046,-3.057110,0,0
23572,0,15,0.04,14,15,-3.429688,-3.135931,0,0
24019,0,13,0.04,10,13,-3.744973,-3.293573,0,0
18378,0,13,0.04,11,13,-3.666152,-3.293573,0,0
...,...,...,...,...,...,...,...,...,...
40292,1,98,1.21,98,114,3.191292,4.667368,1,1
28577,1,99,1.21,99,114,3.270114,4.667368,1,1
97213,1,99,1.21,99,115,3.270114,4.746189,1,0
68116,1,102,1.22,102,118,3.506577,4.982653,1,1


In [8]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [9]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','B_p','ml_outcomes']].groupby(['R','T','B_p']).sum().reset_index()

In [10]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','B_p']].groupby(['R','T','B_p']).count().reset_index()
df_count.columns = ['R','T','B_p','Count']
df_count['N'] = df_count['Count']

In [11]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [12]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,B_p,ml_outcomes,N
2,345,2,0,16,0,0.04,1
3,344,3,0,17,0,0.05,1
13,343,13,0,22,1,0.07,1
5,342,5,0,18,1,0.10,2
0,341,0,0,13,0,0.12,3
...,...,...,...,...,...,...,...
86,4,86,0,59,1,654.24,1128
92,3,92,0,62,1,659.84,1031
98,2,98,0,65,1,667.92,968
90,1,90,0,61,1,673.94,1087


### Setup optimization problem 

In [13]:
from ortools.linear_solver import pywraplp


In [14]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [15]:
dff

,level_0,index,R,T,B_p,ml_outcomes,N
0,341,0,0,13,0,0.12,3
1,340,1,0,15,0,0.12,3
2,345,2,0,16,0,0.04,1
3,344,3,0,17,0,0.05,1
4,329,4,0,18,0,0.50,10
...,...,...,...,...,...,...,...
341,300,341,1,98,1,1.21,1
342,299,342,1,99,0,1.21,1
343,298,343,1,99,1,1.21,1
344,297,344,1,102,1,1.22,1


In [16]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['B_p'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [17]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [18]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [19]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [20]:
## Make sure that you have to add all people in B_p stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    constrain_bp = solver.Constraint(0.0, 0.0)
    
    var1 = vars_cache[(row['R'],row['T'],row['B_p'])]
    key2 = (row['R'],row['T'], 1-row['B_p'])
    
    if key2 not in vars_cache:
        didntexist+=1
        continue
        
    var2 = vars_cache[key2]
    
    constrain_bp.SetCoefficient(var1, -1.0)
    constrain_bp.SetCoefficient(var2, 1.0)
    exists+=1

didntexist, exists

(40, 306)

In [21]:
white_pass_boards = []
white_fail_boards = []
black_pass_boards = []
black_fail_boards = []

for key in vars_cache:
    r, t, b_p = key
    if b_p == 1 and r==0:
        white_pass_boards.append(key)
    elif b_p == 0 and r==0:
        white_fail_boards.append(key)
    elif b_p == 1 and r==1:
        black_pass_boards.append(key)
    elif b_p == 0 and r==1:
        black_fail_boards.append(key)

len(white_pass_boards),len(white_fail_boards),len(black_pass_boards),len(black_fail_boards)

(97, 87, 83, 79)

In [22]:
NUM_TOTALS = {}
df_totals = dff[['N','R','B_p']].groupby(['R','B_p']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['B_p'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['B_p'])] = row['N']

In [23]:
# Now we have one constraint
solver.NumConstraints()

347

In [24]:
#Of those who pass the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_pass_boards = solver.Constraint(0.0, 0.0)

for key in white_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [25]:
#Of those who fail the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards = solver.Constraint(0.0, 0.0)

for key in white_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Solve linear program

In [26]:
solver.ABNORMAL

4

In [27]:
status = solver.Solve()


In [28]:
status

0

In [29]:
solver.OPTIMAL

0

In [30]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [31]:
df_decisions

,row_id,decision
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
341,341,1.0
342,342,1.0
343,343,1.0
344,344,1.0


In [32]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
94,0,94,0,63,1,689.70,1045,94,1.0
90,1,90,0,61,1,673.94,1087,90,1.0
98,2,98,0,65,1,667.92,968,98,1.0
92,3,92,0,62,1,659.84,1031,92,1.0
86,4,86,0,59,1,654.24,1128,86,1.0
...,...,...,...,...,...,...,...,...,...
0,341,0,0,13,0,0.12,3,0,0.0
5,342,5,0,18,1,0.10,2,5,0.0
13,343,13,0,22,1,0.07,1,13,0.0
3,344,3,0,17,0,0.05,1,3,0.0


In [33]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
184,334,184,1,7,0,0.27,1,184,0.0
185,335,185,1,10,0,0.27,1,185,0.0
186,336,186,1,11,0,0.27,1,186,0.0
187,322,187,1,13,0,0.84,3,187,0.0
0,341,0,0,13,0,0.12,3,0,0.0
...,...,...,...,...,...,...,...,...,...
179,286,179,0,113,1,1.98,2,179,1.0
180,314,180,0,115,1,0.99,1,180,1.0
182,310,182,0,123,1,1.00,1,182,1.0
181,311,181,0,123,0,1.00,1,181,1.0


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
94,0,94,0,63,1,689.70,1045,94,1.0
90,1,90,0,61,1,673.94,1087,90,1.0
98,2,98,0,65,1,667.92,968,98,1.0
92,3,92,0,62,1,659.84,1031,92,1.0
86,4,86,0,59,1,654.24,1128,86,1.0
...,...,...,...,...,...,...,...,...,...
0,341,0,0,13,0,0.12,3,0,0.0
5,342,5,0,18,1,0.10,2,5,0.0
13,343,13,0,22,1,0.07,1,13,0.0
3,344,3,0,17,0,0.05,1,3,0.0


In [35]:
xxx.to_csv('./decision.csv')

In [36]:
solution = pd.read_csv('./decision.csv')

In [37]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
94,0,94,0,63,1,689.70,1045,94,1.0
90,1,90,0,61,1,673.94,1087,90,1.0
98,2,98,0,65,1,667.92,968,98,1.0
92,3,92,0,62,1,659.84,1031,92,1.0
86,4,86,0,59,1,654.24,1128,86,1.0
...,...,...,...,...,...,...,...,...,...
0,341,0,0,13,0,0.12,3,0,0.0
5,342,5,0,18,1,0.10,2,5,0.0
13,343,13,0,22,1,0.07,1,13,0.0
3,344,3,0,17,0,0.05,1,3,0.0


In [38]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['B_p'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['B_p'].astype(str)

In [39]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
94,0,94,0,63,1,689.70,1045,94,1.0,0_63_1
90,1,90,0,61,1,673.94,1087,90,1.0,0_61_1
98,2,98,0,65,1,667.92,968,98,1.0,0_65_1
92,3,92,0,62,1,659.84,1031,92,1.0,0_62_1
86,4,86,0,59,1,654.24,1128,86,1.0,0_59_1
...,...,...,...,...,...,...,...,...,...,...
0,341,0,0,13,0,0.12,3,0,0.0,0_13_0
5,342,5,0,18,1,0.10,2,5,0.0,0_18_1
13,343,13,0,22,1,0.07,1,13,0.0,0_22_1
3,344,3,0,17,0,0.05,1,3,0.0,0_17_0


In [40]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_BLACK_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()

In [41]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Equalized Odds',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()

In [42]:
xxx[xxx['R']==0].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
2,345,2,0,16,0,0.04,1,2,0.0,0_16_0
19,252,19,0,26,0,7.02,78,19,0.0,0_26_0
43,125,43,0,38,0,154.98,738,43,0.0,0_38_0
30,259,30,0,31,1,5.33,41,30,0.0,0_31_1
56,123,56,0,44,1,156.30,521,56,0.0,0_44_1
...,...,...,...,...,...,...,...,...,...,...
136,133,136,0,84,1,131.95,145,136,1.0,0_84_1
111,137,111,0,72,0,127.20,159,111,1.0,0_72_0
140,143,140,0,86,1,109.48,119,140,1.0,0_86_1
134,113,134,0,83,1,181.80,202,134,1.0,0_83_1


In [43]:
xxx[xxx['R']==1].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
248,30,248,1,46,0,471.42,873,248,0.0,1_46_0
221,212,221,1,32,1,22.57,61,221,0.0,1_32_1
223,209,223,1,33,1,25.08,66,223,0.0,1_33_1
204,205,204,1,24,0,27.20,85,204,0.0,1_24_0
225,199,225,1,34,1,30.42,78,225,0.0,1_34_1
...,...,...,...,...,...,...,...,...,...,...
311,186,311,1,77,1,43.87,41,311,1.0,1_77_1
294,183,294,1,69,0,46.08,48,294,1.0,1_69_0
275,90,275,1,59,1,242.58,311,275,1.0,1_59_1
308,231,308,1,76,0,13.78,13,308,1.0,1_76_0


In [44]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.5000000000000001

In [45]:
admit_decisions[admit_decisions['B_p']==1][['decision','R']].groupby(['R']).mean()

,decision
R,
0,0.676456
1,0.676456
